In [58]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB


# DATA SETUP
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')
# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
# the min prodcution is a fraction of the max
frac = 0.75


In [59]:
# MODEL
m = gp.Model("transportation")
x = m.addVars(production, distribution, name = "prod_ship")

can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = "can_produce")
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = "must_produce")
meet_demand = m.addConstrs((x.sum('*', d) >= n_demand[d] for d in distribution), name = "meet_demand")

m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

In [ ]:
m.optimize()
x_values = pd.Series(m.getAttr("X", x), name='shipment', index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
obj = m.getObjective().getValue()

print(f"Optimal solution: {obj}")
sol[sol.shipment > 0]

In [61]:
# MODEL 2
M = max_prod# max production
C = 30 # min ship

m = gp.Model("transportation1")
x = m.addVars(production, distribution, name = "prod_ship")
z = m.addVars(production, distribution, vtype=GRB.BINARY, name='ship_active')

can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = "can_produce")
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = "must_produce")
meet_demand = m.addConstrs((x.sum('*', d) >= n_demand[d] for d in distribution), name = "meet_demand")
# x <= 0 or x >= c  
# z1 or z2
# z1 -> x <= M*z1
# z2 -> x + M*z2 >= c  
# z1+z2 >= 1
# z1+z2 = 1 -> z1 = 1 - z2
# a --> x <= M*(1-z)
# b --> x + M*z >= c
min_ship_a = m.addConstrs((x[p,d] <= M[p]*(1-z[p,d]) for p in production for d in distribution), name='min_ship_a')
min_ship_b = m.addConstrs((x[p,d] + M[p]*z[p,d] >= C for p in production for d in distribution), name='min_ship_b')

m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

 

In [ ]:

m.optimize()
x_values = pd.Series(m.getAttr("X", x), name='shipment', index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
obj = m.getObjective().getValue()

print(f"Optimal solution: {obj}")
sol[sol.shipment > 0]

In [ ]:
###################################
## USANDO GLI INDICATOR CONSTRAINTS
###################################
m.addConstrs(((z[p,d] == 1) >> (x[p,d] >= C) for p in production for d in distribution), name='')
m.addConstrs(((z[p,d] == 0) >> (x[p,d] <= 0) for p in production for d in distribution), name='')

In [64]:
###################################
## SEMI CONTINUOUS DECISION VARIABLE 
###################################
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb=C, name='prod_ship')


In [65]:
# ESEMPIO DUE - LIMITAZIONE FACILITIES

In [82]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB


# DATA SETUP
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')
# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
# the min prodcution is a fraction of the max
frac = 0.75

max_prod2 = pd.Series([210,225,140,130,220], index = production, name = "max_production")

m = gp.Model("transportation")
x = m.addVars(production, distribution, name = "prod_ship")

can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod2[p] for p in production), name = "can_produce")
meet_demand = m.addConstrs((x.sum('*', d) >= n_demand[d] for d in distribution), name = "meet_demand")

m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

m.optimize()
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x_values], axis=1)
sol2[sol2.shipment > 0]

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 30 columns and 60 nonzeros
Model fingerprint: 0xa467002a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 2e+02]
Presolve time: 0.00s
Presolved: 11 rows, 30 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.030000e+02   0.000000e+00      0s
       8    1.5984100e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.598410000e+03


cost  shipment
production  distribution                
Baltimore   Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53     101.0
            St. Louis     4.01      29.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      12.0

In [83]:
y = m.addVars(production, vtype=GRB.BINARY, name='activation') # indico se fabbrica è attiva o meno

# if sum(x[p,d]) <= 0 -> y[p] = 0
#  sum(x[p,d] for d in distribution) <= y[p]*M 
m.addConstrs((x.sum(p, '*') <= y[p]*max_prod2[p] for p in production), name='pp')


{'Baltimore': <gurobi.Constr *Awaiting Model Update*>,
 'Cleveland': <gurobi.Constr *Awaiting Model Update*>,
 'Little Rock': <gurobi.Constr *Awaiting Model Update*>,
 'Birmingham': <gurobi.Constr *Awaiting Model Update*>,
 'Charleston': <gurobi.Constr *Awaiting Model Update*>}

In [86]:
# se una fabbrica a cherleston è aperta allora quella a cleveland e baltimore sono chiuse
reg_cond = m.addConstr((y['Cleveland']+y['Baltimore'] <=  (1-y['Charleston'])*2), name='pp')
m.update()
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 35 columns and 101 nonzeros
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 7 rows and 4 columns
Presolve time: 0.00s
Presolved: 11 rows, 31 columns, 63 nonzeros
Variable types: 30 continuous, 1 integer (1 binary)
Found heuristic solution: objective 1878.9800000

Root relaxation: objective 1.877160e+03, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1877.16000    

In [87]:
x_values = pd.Series(m.getAttr('x', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x_values], axis=1)
sol2[sol2.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Lexington     4.33      92.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54      29.0
            St. Louis     4.88      12.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53     101.0
            St. Louis     4.01      29.0